In [8]:
import numpy as np
from scipy.optimize import fsolve
import matplotlib.pyplot as plt
from matplotlib.patches import FancyArrowPatch

def compute_average_S(K, n, b=3):
    exp = np.exp
    W3 = exp(2 * K + K / (2 ** n))  # +++ config
    W1a = exp(-K / (2 ** n))        # ++- or -++ configs
    W1b = exp(-2 * K + K / (2 ** n))  # +-+ config
    Z = W3 + 2 * W1a + W1b
    S_weighted = 3 * W3 + 1 * (2 * W1a + W1b)
    average_S = S_weighted / Z
    return average_S

def rg_step(K, n, b=3):
    avg_S = compute_average_S(K, n, b)
    K_next = K * (avg_S ** 2) / (b ** n)
    return K_next

def find_fixed_point(n, b=3, guess=1.0):
    # Solve for K where <S>(K,n)^2 == b**n (non-trivial fixed point)
    def equation(K):
        return compute_average_S(K, n, b)**2 - b**n
    
    # Check if possible: max <S>^2 =9, so if b**n >9, no solution
    if b**n > 9:
        return None
    K_c, = fsolve(equation, guess)
    if K_c > 0 and abs(equation(K_c)) < 1e-6:
        return K_c
    return None

def plot_rg_flow(n, K_min=0, K_max=2, num_points=20, arrow_length=0.1, b=3):
    fig, ax = plt.subplots(figsize=(10, 2))
    ax.plot([K_min, K_max], [0, 0], 'k-', linewidth=2)  # Axis line
    ax.set_xlim(K_min, K_max)
    ax.set_ylim(-1, 1)
    ax.set_yticks([])
    ax.set_xlabel('K')
    ax.set_title(f'RG Flow for n={n}')
    
    # Find critical point
    K_c = find_fixed_point(n, b)
    if K_c is not None:
        ax.plot(K_c, 0, 'r*', markersize=15, label='Critical Point')
    
    # Sample points and draw arrows
    K_samples = np.linspace(K_min + 0.05, K_max - 0.05, num_points)
    for K in K_samples:
        K_next = rg_step(K, n, b)
        direction = np.sign(K_next - K)
        if direction == 0:
            continue  # Fixed point, no arrow
        arrow_style = "->" if direction > 0 else "<-"
        ax.add_patch(FancyArrowPatch((K, 0), (K + direction * arrow_length, 0),
                                     arrowstyle=arrow_style, mutation_scale=20, color='blue'))
    
    if K_c is not None:
        ax.legend()
    plt.show()

# Generate figures for n=1.5 and n=2.5
plot_rg_flow(1.5)
plot_rg_flow(2.5)

ModuleNotFoundError: No module named 'scipy'